In [10]:
import pandas as pd
df = pd.read_csv('ScrapyParser/kolesa_cars.csv')
df.head()
#df.duplicated()
#len([x for x in df.duplicated() if x == False])

/tmp/ipykernel_138405/764654129.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('ScrapyParser/kolesa_cars.csv')


,manufacturer,model,price,year,body,engine_volume,fuel_type,transmission,city,date
0,Renault,Megane,1 800 000,2004,седан,1.4,бензин,механика,Актобе,29 марта
1,ВАЗ (Lada),2131 (5-ти дверный),1 055 000,2004,внедорожник,1.8,бензин,механика,Актобе,29 марта
2,Chevrolet,Orlando,8 000 000,2013,минивэн,1.8,бензин,автомат,Актобе,29 марта
3,Kia,Cee’d,6 300 000,2013,хэтчбек,1.6,бензин,автомат,Актобе,29 марта
4,Toyota,Camry,17 000 000,2018,седан,2.5,бензин,автомат,Актобе,29 марта
